## Imports

In [1]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm
import pandas as pd
import random

## Settings

In [2]:
in_dir = '/home/drevital/obstacles_classification_datasets/rgb_6_balanced'
out_dir = '/home/drevital/obstacles_classification_datasets/shufersal_sep22_a'
sites_dir = '/home/drevital/obstacles_classification_datasets/rgb_6_balanced/sites'
sites = os.listdir(sites_dir)

# Parameters used in the diff_metric to diff_coef assignent function
alfa = -3.5
beta = 2.0
gamma = 8
swc = 2.0 # sample weight coefficient
diff_threshold = 50
std_threshold_dist = 1.5 # Distance from std to apply sample_weight correction

In [3]:
sites

['kfar_saba',
 'koki_factory',
 'new_factory_humid',
 'musashi_office',
 'shufersal',
 'new_factory',
 'neve_ilan',
 'unknown']

In [4]:
site_thresholds = {'neve_ilan': 55,
                   'kfar_saba': 55,
                   'shufersal': 55,
                   'new_factory': 50,
                   'new_factory_humid': 50,
                   'musashi_office': 40,
                   'koki_factory': 40,
                   'unknown': 50}
default_threshold = 50

## Make dictionary for the image names of each site

In [5]:
site_images = defaultdict(list)

for site in sites:
    site_dir = os.path.join(sites_dir, site)
    site_images[site] = os.listdir(site_dir)

In [6]:
site_images.keys()

dict_keys(['kfar_saba', 'koki_factory', 'new_factory_humid', 'musashi_office', 'shufersal', 'new_factory', 'neve_ilan', 'unknown'])

## A funciton to find the source site of a given image

In [7]:
def find_site_and_threshold(im_name):
    found_states = [im_name in site_images[site] for site in sites]
    
    if any(found_states):
        site = sites[np.argmax(found_states)]
        threshold = site_thresholds[site]
    else:
        site = 'unknown'
        site_images[site].append(im_name)
        threshold = default_threshold
        
    return site, threshold

## Define curve to assign diff_coef according to diff_metric

In [8]:
def diff_metric_to_diff_coef(sigma_dist):
    
    # Correction curve for assigning coefficients
    # Based on Sigmoid
    # adding alpha, beta and gamma controls, as explained at the
    # beginning of this notebook
    
    return 1/(1 + np.exp(-(sigma_dist*alfa-beta)*gamma))

## Calculate sample_weights

In [9]:
train_dict = {'in_name': [],
              'out_name': [],
              'class_name': [],
              'diff_metric': [],
              'diff_coef': [],
              'sample_weight': []
             }
diff_metrics = {'no_obstacle': [], 'obstacle': []}
class_names = ['no_obstacle', 'obstacle']
subset_name = 'train'

for class_name in class_names:
    class_path = os.path.join(in_dir, subset_name, class_name)
    im_names = os.listdir(class_path)
    for im_name in tqdm(im_names):
        im_path = os.path.join(class_path, im_name)
        pair = cv2.imread(im_path)

        # Generate diff mask            
        w = pair.shape[1]
        ref = pair[:, :w//2]
        current = pair[:, w//2:(w//2)*2]
        diff = cv2.subtract(ref, current)
        agg_rgb = np.stack((diff[:, :, 0], diff[:, :, 1], diff[:, :, 2])).max(0)
        _, mask = cv2.threshold(agg_rgb, diff_threshold, 255, cv2.THRESH_BINARY)

        # Calculate diff_coeff
        h = mask.shape[0]
        w = mask.shape[1]
        area = h * w

        # Update train dictionary
        train_dict['in_name'].append(im_name)
        train_dict['class_name'].append(class_name)
        diff_metric = (np.sum(mask)/255)/area
        train_dict['diff_metric'].append(diff_metric)    
        diff_metrics[class_name].append(diff_metric)
            
mean = {'no_obstacle': np.mean(diff_metrics['no_obstacle']),
        'obstacle': np.mean(diff_metrics['obstacle'])}
std = {'no_obstacle': np.std(diff_metrics['no_obstacle']),
       'obstacle': np.std(diff_metrics['obstacle']) }

for i, diff_metric in enumerate(train_dict['diff_metric']):
    class_name = train_dict['class_name'][i]
    # Following is to adjust the direction of distance from std and correction accordingly
    # For obstacle - a negative sigma means we are lower than threshold and need correction
    # For no obstacle a positive sigma means we are higher than threshold and need correction
    sigma_dist_sign = 1.0 if class_name == 'obstacle' else -1.0 
    diff_threshold = mean[class_name] + sigma_dist_sign * std_threshold_dist * std[class_name]
    sigma_dist = sigma_dist_sign * (diff_metric - diff_threshold)/std[class_name]
    diff_coef = diff_metric_to_diff_coef(sigma_dist)
    sample_weight = 1.0 + swc * diff_coef
    train_dict['diff_coef'].append(diff_coef)
    train_dict['sample_weight'].append(sample_weight)

100%|██████████| 12288/12288 [00:35<00:00, 346.77it/s]


## A function to generate <ref, current, mask> triplet from <ref, current> pair

In [10]:
def triplet_image(pair, threshold):
    w = pair.shape[1]
    ref = pair[:, :w//2]
    current = pair[:, w//2:(w//2)*2]
    diff = cv2.absdiff(current, ref)
    agg_rgb = np.stack((diff[:, :, 0], diff[:, :, 1], diff[:, :, 2])).max(0)
    _, mask = cv2.threshold(agg_rgb, threshold, 255, cv2.THRESH_BINARY)

    # old morphological operations
    copyImg = cv2.erode(mask, np.ones((3, 3), np.uint8), iterations=1)  # reduce noise
    copyImg = cv2.dilate(copyImg, np.ones((7, 7), np.uint8), iterations=1)
    copyImg = cv2.erode(copyImg, np.ones((5, 5), np.uint8), iterations=1)
    copyImg = cv2.dilate(copyImg, np.ones((9, 9), np.uint8), iterations=1)
    kernel = np.ones((11, 11), np.uint8)  # kernel for dilation

    # increase area to an object
    copyImg = cv2.dilate(copyImg, kernel, iterations=2)
    copyImg = cv2.dilate(copyImg, np.ones((13, 13), np.uint8), iterations=1)
    copyImg = cv2.erode(copyImg, np.ones((11, 11), np.uint8), iterations=1)
    copyImg = cv2.erode(copyImg, np.ones((5, 5), np.uint8), iterations=1)

    mask = copyImg 
    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)  
    
    return cv2.hconcat([ref, current, mask])

## Generate triplet images <ref, current, mask>

In [11]:
subset_names = ['train', 'eval']
class_names = ['no_obstacle', 'obstacle']
class_extensions = {'no_obstacle': 'noobs', 'obstacle': 'obs'}

for subset_name in subset_names:
    cur_out_dir = os.path.join(out_dir, subset_name)
    Path(cur_out_dir).mkdir(parents=True, exist_ok=True)

# Prepare indices for shuffling the images in dictionary, so onstacle/no_obstacle are mixed
# This is necessary for the training/validation from corresponding dataframe to work properly

keys = list(train_dict.keys())
shuffled_train_dict = {}
inds = [i for i in range(len(train_dict['in_name']))]
shuffled = inds.copy()
random.shuffle(shuffled)
for k in keys:
    if len(train_dict[k]) > 0:
        shuffled_train_dict[k] = [train_dict[k][shuffled[i]] for i in range(len(train_dict['in_name']))]
    else:
        shuffled_train_dict[k] = []
    
subset_name = 'train'
i = 0
for im_name in tqdm(shuffled_train_dict['in_name']):
    class_name = shuffled_train_dict['class_name'][i]
    class_path = os.path.join(in_dir, subset_name, class_name)
    im_path = os.path.join(class_path, im_name)
    pair = cv2.imread(im_path)
    site, threshold = find_site_and_threshold(im_name)
    triplet = triplet_image(pair, threshold)
    class_extension = class_extensions[class_name]
    sample_weight = shuffled_train_dict['sample_weight'][i]
    out_im_name = '.'.join(im_name.split('.')[:-1])\
         + f'_{site}_{class_extension}_{sample_weight:.4f}_.jpg'
    shuffled_train_dict['out_name'].append(out_im_name)
    cur_out_dir = os.path.join(out_dir, subset_name)
    out_path = os.path.join(cur_out_dir, out_im_name)
    cv2.imwrite(out_path, triplet)
    i += 1
    
subset_name = 'eval'
for class_name in class_names:
    class_path = os.path.join(in_dir, subset_name, class_name)
    im_names = os.listdir(class_path)
    cur_out_dir = os.path.join(out_dir, subset_name, class_name)
    Path(cur_out_dir).mkdir(parents=True, exist_ok=True)
    for im_name in tqdm(im_names):
        im_path = os.path.join(class_path, im_name)
        pair = cv2.imread(im_path)
        site, threshold = find_site_and_threshold(im_name)
        triplet = triplet_image(pair, threshold)
        out_im_name = '.'.join(im_name.split('.')[:-1]) + f'_{site}_.jpg'
        out_path = os.path.join(cur_out_dir, out_im_name)
        cv2.imwrite(out_path, triplet)    

100%|██████████| 1282/1282 [00:08<00:00, 150.57it/s]


## Create Dataframe from sample_weights Dictionary

In [12]:
shuffled_train_dict.keys()

dict_keys(['in_name', 'out_name', 'class_name', 'diff_metric', 'diff_coef', 'sample_weight'])

In [13]:
[len(shuffled_train_dict[k]) for k in shuffled_train_dict.keys()]

[24152, 24152, 24152, 24152, 24152, 24152]

In [14]:
train_df = pd.DataFrame.from_dict(shuffled_train_dict)

In [15]:
train_df

,in_name,out_name,class_name,diff_metric,diff_coef,sample_weight
0,759_98942_.76_2022-02-01--03-53-16_.jpg,759_98942_.76_2022-02-01--03-53-16__unknown_ob...,obstacle,0.174179,1.000000e+00,3.000000
1,946_206412_.63_2022-02-01--06-37-52_.jpg,946_206412_.63_2022-02-01--06-37-52__unknown_n...,no_obstacle,0.029762,9.999983e-01,2.999997
2,29_136001_.69_2022-02-01--04-49-33_.jpg,29_136001_.69_2022-02-01--04-49-33__unknown_ob...,obstacle,0.053016,1.000000e+00,3.000000
3,1_473_.99_2021-08-15--17-13-12_.jpg,1_473_.99_2021-08-15--17-13-12__new_factory_ob...,obstacle,0.144259,1.000000e+00,3.000000
4,1_488_.95_2021-08-15--15-01-35_.jpg,1_488_.95_2021-08-15--15-01-35__new_factory_ob...,obstacle,0.320576,7.774061e-07,1.000002
...,...,...,...,...,...,...
24147,9_420_.49_2021-08-17--17-59-07_.jpg,9_420_.49_2021-08-17--17-59-07__musashi_office...,no_obstacle,0.003755,9.990013e-01,2.998003
24148,128_128492_.86_2022-02-01--04-38-02_.jpg,128_128492_.86_2022-02-01--04-38-02__unknown_n...,no_obstacle,0.045724,1.000000e+00,3.000000
24149,122_571__contrast_758.jpg,122_571__contrast_758_unknown_noobs_2.9958_.jpg,no_obstacle,0.000718,9.979056e-01,2.995811
24150,120_2185_.93_2021-08-16--15-32-04_.jpg,120_2185_.93_2021-08-16--15-32-04__neve_ilan_o...,obstacle,0.053001,1.000000e+00,3.000000


## Store the Dataframe in a .csv file

In [16]:
csv_name = out_dir.split('/')[-1] + '.csv'
csv_path = os.path.join(out_dir, csv_name)
train_df.to_csv(csv_path)

In [17]:
mean['obstacle'], std['obstacle']

(0.16480831052568698, 0.10885395623125917)

In [18]:
mean['no_obstacle'], std['no_obstacle']

(0.08192730783396576, 0.11464716390375307)

In [22]:
tot = 0
for site in site_images:
    tot += len(site_images[site])
    print(f'{site}: {len(site_images[site])}')
print(f'========== total: {tot}')

kfar_saba: 1483
koki_factory: 437
new_factory_humid: 378
musashi_office: 3171
shufersal: 798
new_factory: 7649
neve_ilan: 3486
unknown: 10271
========== total: 27673
